In [0]:
import pandas as pd
df=pd.read_csv('med_data.csv', encoding='cp1252')
df.head()

FileNotFoundError: ignored

In [0]:
print(len(df))
df = df[df['Quantity']>0]
print(len(df))

NameError: ignored

In [0]:
df_t = df[['ProductType','Quantity','Date']]
df_t = df_t.groupby(['ProductType','Date']).sum()
df_t = df_t.reset_index()
print(len(df_t))
df_t.info()

In [0]:
df_con = df_t[df_t['ProductType']=='CON']
df_con = df_con.drop(['ProductType'],axis=1)
df_con["Date"] = pd.to_datetime(df_t['Date'])
df_con = df_con.sort_values(by=['Date'])
df_con = df_con.set_index('Date')
df_con = df_con.rename(columns={'Quantity':'x'})
print(len(df_con))
df_con.head()

In [0]:
import matplotlib.pyplot as plt 
rolmean = df_con.rolling(14).mean()
plt.plot(rolmean, color='red', label='Rolling Mean')
plt.figure(figsize=(20,10))
plt.xlabel('Date')
plt.ylabel('Quantity of Chemicals Sold')
plt.plot(df_con)
plt.show()

In [0]:
from statsmodels.tsa.seasonal import seasonal_decompose


original = df_con

decomposition = seasonal_decompose(original, freq=14)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.figure(figsize=(25,12))

plt.subplot(411)
plt.plot(original, label="Original")
plt.ylabel('Quantity of Chemicals Sold')
plt.legend(loc='best')

plt.subplot(412)
plt.plot(trend, label="Trend")
plt.ylabel('Quantity of Chemicals Sold')
plt.legend(loc='best')


plt.subplot(413)
plt.plot(seasonal, label="Seasonal")
plt.ylabel('Quantity of Chemicals Sold')
plt.legend(loc='best')


plt.subplot(414)
plt.plot(residual, label="Residual")
plt.xlabel('Date')
plt.ylabel('Quantity of Chemicals Sold')
plt.legend(loc='best')

In [0]:
from fbprophet import Prophet
df = trend.reset_index()
df.columns = ['ds', 'y']
m_original = Prophet()
m_original.fit(df)
future_original = m_original.make_future_dataframe(periods=90)
future_original = m_original.predict(future_original)
forcasted_original = future_original[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forcasted_original.ds = pd.to_datetime(forcasted_original.ds)
forcasted_original = forcasted_original.set_index('ds')
plt.figure(figsize=(20,10))
plt.plot(forcasted_original.index, forcasted_original.yhat)

In [0]:
from fbprophet.diagnostics import cross_validation
df_cv = cross_validation(m_original, initial='180 days', period='60 days', horizon = '90 days')
df_cv.head()

In [0]:
from fbprophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.tail()